## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 13.3 ms, sys: 8.28 ms, total: 21.5 ms
Wall time: 20.8 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 32.8 s, sys: 123 ms, total: 33 s
Wall time: 32.7 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [16]:

# this is very quick, so we can use grid search on these models.

run_logfile = "runs2bal.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96.96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal",
                      model_dir        = "../models/CNNBal")
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 7ms/step - loss: 0.6347 - accuracy: 0.6403 - val_loss: 0.5286 - val_accuracy: 0.7382
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.4349 - accuracy: 0.8015 - val_loss: 0.3914 - val_accuracy: 0.8336
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2867 - accuracy: 0.8842 - val_loss: 0.3490 - val_accuracy: 0.8621
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1837 - accuracy: 0.9282 - val_loss: 0.2333 - val_accuracy: 0.9168
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1145 - accuracy: 0.9608 - val_loss: 0.2387 - val_accuracy: 0.9218
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0761 - accuracy: 0.9739 - val_loss: 0.3378 - val_accuracy: 0.8951
Epoch 7/20
288/288 [=====================

288/288 [==============================] - 1s 5ms/step - loss: 0.0117 - accuracy: 0.9967 - val_loss: 0.4418 - val_accuracy: 0.9272
Epoch 16/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0138 - accuracy: 0.9963 - val_loss: 0.5121 - val_accuracy: 0.9159
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0118 - accuracy: 0.9965 - val_loss: 0.7724 - val_accuracy: 0.8797
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0152 - accuracy: 0.9957 - val_loss: 0.4011 - val_accuracy: 0.9340
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0157 - accuracy: 0.9956 - val_loss: 0.2409 - val_accuracy: 0.9580
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0169 - accuracy: 0.9932 - val_loss: 0.4282 - val_accuracy: 0.9254



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: 

288/288 [==============================] - 2s 5ms/step - loss: 0.0225 - accuracy: 0.9937 - val_loss: 0.4418 - val_accuracy: 0.8987
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0219 - accuracy: 0.9926 - val_loss: 0.2161 - val_accuracy: 0.9543
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0214 - accuracy: 0.9941 - val_loss: 0.3877 - val_accuracy: 0.9231
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0110 - accuracy: 0.9972 - val_loss: 0.7269 - val_accuracy: 0.8531
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0168 - accuracy: 0.9949 - val_loss: 0.5077 - val_accuracy: 0.9064
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0132 - accuracy: 0.9950 - val_loss: 0.3460 - val_accuracy: 0.9263
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0126 - accuracy: 0.9965 - val_loss: 0.2864 - val_accuracy: 0.9480
Epoch 15/20


288/288 [==============================] - 2s 7ms/step - loss: 0.6412 - accuracy: 0.6549 - val_loss: 0.5093 - val_accuracy: 0.7645
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.3856 - accuracy: 0.8315 - val_loss: 0.2513 - val_accuracy: 0.9010
Epoch 3/20
288/288 [==============================] - 2s 5ms/step - loss: 0.2279 - accuracy: 0.9119 - val_loss: 0.1989 - val_accuracy: 0.9236
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1210 - accuracy: 0.9618 - val_loss: 0.1655 - val_accuracy: 0.9385
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0755 - accuracy: 0.9781 - val_loss: 0.1502 - val_accuracy: 0.9534
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0498 - accuracy: 0.9857 - val_loss: 0.1948 - val_accuracy: 0.9417
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0269 - accuracy: 0.9931 - val_loss: 0.3103 - val_accuracy: 0.9123
Epoch 8/20
288/28

Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0112 - accuracy: 0.9962 - val_loss: 0.4480 - val_accuracy: 0.9241
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0088 - accuracy: 0.9974 - val_loss: 0.5134 - val_accuracy: 0.9141
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0095 - accuracy: 0.9973 - val_loss: 0.6303 - val_accuracy: 0.8974
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0096 - accuracy: 0.9966 - val_loss: 0.9115 - val_accuracy: 0.8757
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0114 - accuracy: 0.9961 - val_loss: 0.1898 - val_accuracy: 0.9638



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6779 - accuracy: 0.60

...

288/288 [==============================] - 3s 8ms/step - loss: 0.6481 - accuracy: 0.6181 - val_loss: 0.6257 - val_accuracy: 0.6704
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.4641 - accuracy: 0.7869 - val_loss: 0.3726 - val_accuracy: 0.8486
Epoch 3/20
288/288 [==============================] - 2s 7ms/step - loss: 0.3021 - accuracy: 0.8823 - val_loss: 0.3375 - val_accuracy: 0.8725
Epoch 4/20
288/288 [==============================] - 2s 7ms/step - loss: 0.2031 - accuracy: 0.9229 - val_loss: 0.2992 - val_accuracy: 0.8947
Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.1355 - accuracy: 0.9527 - val_loss: 0.3820 - val_accuracy: 0.8689
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0858 - accuracy: 0.9723 - val_loss: 0.2676 - val_accuracy: 0.9186
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0760 - accuracy: 0.9748 - val_loss: 0.3493 - val_accuracy: 0.9082
Epoch 8/20
288/28

288/288 [==============================] - 2s 7ms/step - loss: 0.0117 - accuracy: 0.9956 - val_loss: 0.4005 - val_accuracy: 0.9245
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.3192 - val_accuracy: 0.9412
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.4295 - val_accuracy: 0.9254
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0073 - accuracy: 0.9978 - val_loss: 0.2818 - val_accuracy: 0.9435
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.3802 - val_accuracy: 0.9231
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0088 - accuracy: 0.9976 - val_loss: 0.3842 - val_accuracy: 0.9286
Epoch 20/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0045 - accuracy: 0.9989 - val_loss: 0.2788 - val_accuracy: 0.9471



########

288/288 [==============================] - 2s 7ms/step - loss: 0.0427 - accuracy: 0.9906 - val_loss: 0.2038 - val_accuracy: 0.9331
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0199 - accuracy: 0.9951 - val_loss: 0.2271 - val_accuracy: 0.9304
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0110 - accuracy: 0.9983 - val_loss: 0.1485 - val_accuracy: 0.9571
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0075 - accuracy: 0.9984 - val_loss: 0.2619 - val_accuracy: 0.9322
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0055 - accuracy: 0.9993 - val_loss: 0.2876 - val_accuracy: 0.9304
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0026 - accuracy: 0.9998 - val_loss: 0.3067 - val_accuracy: 0.9322
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.3532 - val_accuracy: 0.9299
Epoch 12/20
288

288/288 [==============================] - 2s 7ms/step - loss: 7.6476e-04 - accuracy: 0.9999 - val_loss: 0.3312 - val_accuracy: 0.9453
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.3104 - val_accuracy: 0.9575
Epoch 20/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 0.8721 - val_accuracy: 0.8892



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 128
Adding Convolution: Kernel Size: 8, Filter Count: 128
Adding Convolution: Kernel Size: 32, Filter Count: 128
Adding Convolution: Kernel Size: 64, Filter Count: 128
Epoch 1/20
288/288 [==============================] - 3s 9ms/step - loss: 0.6319 - accuracy: 0.6366 - val_loss: 0.3586 - val_accuracy: 0.8391
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.3353 - accuracy: 0.8559 - val_loss: 0.2528 - val_accuracy: 0.8

288/288 [==============================] - 2s 7ms/step - loss: 0.0234 - accuracy: 0.9931 - val_loss: 0.1813 - val_accuracy: 0.9589
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0132 - accuracy: 0.9961 - val_loss: 0.3786 - val_accuracy: 0.9141
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0183 - accuracy: 0.9950 - val_loss: 0.4336 - val_accuracy: 0.9132
Epoch 12/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0176 - accuracy: 0.9949 - val_loss: 0.3795 - val_accuracy: 0.9195
Epoch 13/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0142 - accuracy: 0.9957 - val_loss: 0.2045 - val_accuracy: 0.9607
Epoch 14/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0063 - accuracy: 0.9990 - val_loss: 0.2099 - val_accuracy: 0.9589
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0119 - accuracy: 0.9955 - val_loss: 0.5823 - val_accuracy: 0.8969
Epoch 16/20





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/